# Standalone Data Valiation with Great_Expecations in a Notebook

In [1]:
import numpy as np
import pandas as pd
import os 
import great_expectations as ge
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 1. Loading data

### 1.1 from local csv file

In [2]:
dx_df = ge.read_csv(
    "../../rwd_data_quality/data/diagnosis.csv",
     sep="\t")
dx_df.head()
dx_df.dtypes

,pt_id,dos,dx_code
0,13151,2018-04-10,H35.3231
1,13151,2018-05-22,H35.3231
2,13151,2018-07-17,H35.3220
3,13151,2018-09-25,H35.3220
4,13151,2018-12-18,H35.3220


pt_id       int64
dos        object
dx_code    object
dtype: object

In [3]:
pc_df = ge.read_csv(
    "../../rwd_data_quality/data/procedure.csv",
     sep="\t")
pc_df.head()
pc_df.shape
pc_df.dtypes

,patient_id,dos,procedure_code,mod
0,10026,12/14/19,2027F,NaN
1,10026,11/13/19,92014,25
2,10026,11/6/19,G8427,NaN
3,10026,10/2/19,92134,NaN
4,10026,9/23/19,92014,25


(5222, 4)

patient_id         int64
dos               object
procedure_code    object
mod               object
dtype: object

In [4]:
pt_df = ge.read_csv(
    "../../rwd_data_quality/data/patient.csv",
     sep="\t")
pt_df.head()
pt_df.shape
pt_df.dtypes

,patientid,sex
0,13151,Female
1,43463,Female
2,42834,Male
3,37531,Female
4,31613,Female


(945, 2)

patientid     int64
sex          object
dtype: object

### 1.2 from Pandas Dataframe or Spark DataFrame

my_df = ge.from_pandas(
    my_pandas_dataframe
)

from great_expectations.dataset.sparkdf_dataset import SparkDFDataset

my_df = SparkDFDataset(my_spark_dataframe)

In [5]:
dx_df.dos.value_counts()

2016-06-29    60
2016-01-08    50
2015-12-21    48
2016-08-16    48
2015-11-23    45
              ..
2019-10-31     1
2018-02-27     1
2016-09-19     1
2019-04-23     1
2017-03-01     1
Name: dos, Length: 685, dtype: int64

## 2. Exploring data and adding expectations 

### 2.1. String matching with Regex

#### 2.1.1. ICD-9-CM 

Most ICD-9 diagnosis codes are just numbers, but they may also start with E or V. 

* Numeric ICD-9 codes are at least three digits. Optionally there may be a decimal followed by one of two more digits.
* An E code begins with E and three digits. These may be followed by a decimal and one more digit.
* A V code begins with a V followed by two digits. These may be followed by a decimal and one or two more digits.

Sometimes the decimals are left out. Here are regular expressions that summarize the discussion above.

In [6]:
N = "\d{3}\.?\d{0,2}"
E = "E\d{3}\.?\d?"
V = "V\d{2}\.?\d{0,2}"
icd9_regex = "|".join([N, E, V])

In [7]:
dx_df.expect_column_values_to_match_regex('dx_code', icd9_regex)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 4672,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1632,
    "unexpected_percent": 34.93150684931507,
    "unexpected_percent_total": 34.93150684931507,
    "unexpected_percent_nonmissing": 34.93150684931507,
    "partial_unexpected_list": [
      "H35.32",
      "H35.32",
      "H34.12",
      "H35.81",
      "H35.32",
      "H35.81",
      "H25.13",
      "H25.13",
      "H35.32",
      "H43.13",
      "H35.32",
      "E78.00",
      "H35.32",
      "H35.32",
      "H35.32",
      "H35.32",
      "H35.32",
      "H35.32",
      "E11.9",
      "H35.32"
    ]
  },
  "success": false
}

#### 2.1.2. Regular Expression for ICD-10-CM Code 
ICD-10 diagnosis codes always begin with a letter (except U) followed by a digit. The third character is usually a digit, but could be an A or B. After the first three characters, there may be a decimal point, and up to three more alphanumeric characters. These alphanumeric characters are never U. Sometimes the decimal is left out. So the following regular expression would match any ICD-10 diagnosis code.

In [8]:
icd10_regex=r"(?i)[A-TV-Z][0-9][0-9AB]\.?[0-9A-TV-Z]{0,4}"

In [9]:
dx_df.expect_column_values_to_match_regex(
    'dx_code', 
    icd10_regex
)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 4672,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1460,
    "unexpected_percent": 31.25,
    "unexpected_percent_total": 31.25,
    "unexpected_percent_nonmissing": 31.25,
    "partial_unexpected_list": [
      "312912001",
      "312912001",
      "36252",
      "362.52",
      "362.52",
      "312912001",
      "362.52",
      "362.52",
      "362.52",
      "362.52",
      "362.52",
      "362.52",
      "362.52",
      "362.52",
      "362.52",
      "362.52",
      "362.02",
      "362.02",
      "362.02",
      "362.02"
    ]
  },
  "success": false
}

#### 2.1.3. PCT, HCPCS code Format

#### 2.1.4. Diagnosis codes of a certain disease
[age-related macular degeneration](https://mdinteractive.com/MIPS_ophthalmology)

In [10]:
wamd_re = r"(?i)(H35.?32)|(362.?52)|(414173003)"  #(?i) case insensitive
dx_df.expect_column_values_to_match_regex('dx_code', 
                                          wamd_re, 
                                          mostly=1,
                                          result_format={'result_format': 'BASIC'},
                                         )

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 4672,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 3379,
    "unexpected_percent": 72.32448630136986,
    "unexpected_percent_total": 72.32448630136986,
    "unexpected_percent_nonmissing": 72.32448630136986,
    "partial_unexpected_list": [
      "312912001",
      "312912001",
      "H35.3112",
      "312912001",
      "H43.811",
      "H34.12",
      "H35.81",
      "H02.403",
      "362.02",
      "362.02",
      "362.02",
      "362.02",
      "374.3",
      "H02.403",
      "H43.812",
      "H04.123",
      "H35.81",
      "H25.13",
      "H25.13",
      "H43.813"
    ]
  },
  "success": false
}

### 2.2. Misssing values

In [28]:
dx_df.expect_column_values_to_not_be_null('dx_code', mostly=1)
dx_df.expect_column_values_to_not_be_null('pt_id', mostly=1)
dx_df.expect_column_values_to_not_be_null('dos', mostly=1)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 4672,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 4672,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 4672,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [12]:
pt_df.expect_column_values_to_not_be_null('patientid', mostly=1)
pt_df.expect_column_values_to_not_be_null('sex', mostly=1)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 945,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 945,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [13]:
pc_df.expect_column_values_to_not_be_null('mod', mostly=1)
pc_df.expect_column_values_to_not_be_null('patient_id', mostly=1)
pc_df.expect_column_values_to_not_be_null('dos', mostly=1)
pc_df.expect_column_values_to_not_be_null('procedure_code', mostly=1)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "unexpected_count": 3760,
    "unexpected_percent": 72.00306396016852,
    "unexpected_percent_total": 72.00306396016852,
    "partial_unexpected_list": []
  },
  "success": false
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

### 2.3. Values in a given set

In [14]:
pc_df.expect_column_values_to_be_in_set('procedure_code', ['67028'], mostly=1)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 4716,
    "unexpected_percent": 90.31022596706244,
    "unexpected_percent_total": 90.31022596706244,
    "unexpected_percent_nonmissing": 90.31022596706244,
    "partial_unexpected_list": [
      "2027F",
      "92014",
      "G8427",
      "92134",
      "92014",
      "92134",
      "2026F",
      "1036F",
      "92012",
      "92014",
      "92012",
      "92134",
      "92014",
      "92226",
      "92014",
      "92014",
      "EYLEAX1",
      "99213",
      "92015",
      "92012"
    ]
  },
  "success": false
}

In [15]:
pt_df.expect_column_values_to_be_in_set('sex', ['Male', "Female", 'Not Reported'], mostly=1)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 945,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 24,
    "unexpected_percent": 2.5396825396825395,
    "unexpected_percent_total": 2.5396825396825395,
    "unexpected_percent_nonmissing": 2.5396825396825395,
    "partial_unexpected_list": [
      "female",
      "f",
      "male",
      "female",
      "female",
      "f",
      "female",
      "female",
      "male",
      "male",
      "male",
      "female",
      "male",
      "male",
      "male",
      "male",
      "male",
      "f",
      "M",
      "female"
    ]
  },
  "success": false
}

### 2.4. Values representing a datetime with a given format.

In [16]:
dx_df.expect_column_values_to_match_strftime_format('dos', '%Y-%m-%d')
pc_df.expect_column_values_to_match_strftime_format('dos', '%Y-%m-%d')
pc_df.expect_column_values_to_match_strftime_format('dos', '%x')

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 4672,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 5222,
    "unexpected_percent": 100.0,
    "unexpected_percent_total": 100.0,
    "unexpected_percent_nonmissing": 100.0,
    "partial_unexpected_list": [
      "12/14/19",
      "11/13/19",
      "11/6/19",
      "10/2/19",
      "9/23/19",
      "2/7/19",
      "1/8/19",
      "1/4/19",
      "11/21/18",
      "7/18/18",
      "6/19/18",
      "6/15/18",
      "12/6/17",
      "10/27/17",
      "8/1/17",
      "7/31/17",
      "5/25/17",
      "4/24/17",
      "1/24/17",
      "1/16/17"
    ]
  },
  "success": false
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [17]:
from datetime import datetime as dt
pc_df['year']=pc_df['dos'].apply(lambda x: dt.strptime(x, '%x').year)
pc_df.head()

,patient_id,dos,procedure_code,mod,year
0,10026,12/14/19,2027F,NaN,2019
1,10026,11/13/19,92014,25,2019
2,10026,11/6/19,G8427,NaN,2019
3,10026,10/2/19,92134,NaN,2019
4,10026,9/23/19,92014,25,2019


### 2.5. Duplicate Values

#### 2.5.1. No duplicated column value

In [18]:
pt_df.expect_column_values_to_be_unique(
    'patientid', 
    result_format={'result_format': 'BASIC'}, 
)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 945,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 26,
    "unexpected_percent": 2.751322751322751,
    "unexpected_percent_total": 2.751322751322751,
    "unexpected_percent_nonmissing": 2.751322751322751,
    "partial_unexpected_list": [
      10026,
      25601,
      33971,
      42472,
      16661,
      44456,
      33971,
      34041,
      41540,
      10026,
      39510,
      16661,
      21629,
      44456,
      37952,
      42472,
      24434,
      25601,
      21629,
      34041
    ]
  },
  "success": false
}

#### 2.5.2. No duplicate records, ie compound column values

In [19]:
pc_df.expect_compound_columns_to_be_unique(
    pc_df.columns, 
    result_format={'result_format': 'BASIC'}, 
)
pt_df.expect_compound_columns_to_be_unique(
    pt_df.columns, 
    result_format={'result_format': 'BOOLEAN_ONLY'}, 
)
dx_df.expect_compound_columns_to_be_unique(
    dx_df.columns, 
    result_format={'result_format': 'BOOLEAN_ONLY'}, 
)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 135,
    "unexpected_percent": 2.5852163921869016,
    "unexpected_percent_total": 2.5852163921869016,
    "unexpected_percent_nonmissing": 2.5852163921869016,
    "partial_unexpected_list": [
      {
        "patient_id": 10026,
        "dos": "12/20/16",
        "procedure_code": "67210",
        "mod": "79",
        "year": 2016
      },
      {
        "patient_id": 10026,
        "dos": "12/20/16",
        "procedure_code": "67210",
        "mod": "79",
        "year": 2016
      },
      {
        "patient_id": 10098,
        "dos": "2/27/17",
        "procedure_code": "92134",
        "mod": null,
        "year": 2017
      },
      {
        "patient_id": 10098,
        "dos": "9/27/18",
        "procedure_code": "92226",
      

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {},
  "success": false
}

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {},
  "success": false
}

### 2.6. Multiple entries in a single cell

In [20]:
sep = r"[,\s;]+"
pc_df.expect_column_values_to_not_match_regex('procedure_code', sep, mostly=1)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 8,
    "unexpected_percent": 0.15319800842589046,
    "unexpected_percent_total": 0.15319800842589046,
    "unexpected_percent_nonmissing": 0.15319800842589046,
    "partial_unexpected_list": [
      "J2778,05MG",
      "J9035,J3490,Q9977",
      "J9035,J3490,Q9977",
      "J9035,J3490,Q9977",
      "J9035,J3490,Q9977",
      "J9035,J3490,Q9977",
      "J9035,J3490,Q9977",
      "J9035,J3490,Q9977"
    ]
  },
  "success": false
}

In [21]:
pc_df.expect_column_value_lengths_to_be_between('procedure_code', min_value=4, max_value=5)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 5222,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 124,
    "unexpected_percent": 2.3745691306013024,
    "unexpected_percent_total": 2.3745691306013024,
    "unexpected_percent_nonmissing": 2.3745691306013024,
    "partial_unexpected_list": [
      "EYLEAX1",
      "EYLEAX1",
      "EYLEAX1",
      "EYLEAX1",
      "J2778PF",
      "J2778PF",
      "J2778PF",
      "J2778PF",
      "J2778PF",
      "J2778PF",
      "J2778PF",
      "J2778PF",
      "J2778PF",
      "67028MCR",
      "J0178OU",
      "EYLEAX1",
      "LUC5SYRX1",
      "EYLEAX1",
      "LUC5SYRX1",
      "LUC5SYRX1"
    ]
  },
  "success": false
}

## 3. Creating suite for the automation of data validation

In [22]:
pt_df.get_expectation_suite(discard_failed_expectations=False)

{
  "expectations": [
    {
      "meta": {},
      "kwargs": {
        "column": "patientid",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "sex",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "sex",
        "value_set": [
          "Male",
          "Female",
          "Not Reported"
        ],
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_be_in_set"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "patientid"
      },
      "expectation_type": "expect_column_values_to_be_unique"
    },
    {
      "meta": {},
      "kwargs": {
        "column_list": [
          "patientid",
          "sex"
        ]
      },
      "expectation_type": "expect_compound_columns_to_be_unique"
    }
  ],
  "meta": {
    "great_ex

In [23]:
with open( "pt_expectation_file.json", "w") as pt_file:
    pt_file.write(
        json.dumps(pt_df.get_expectation_suite().to_json_dict())
    )

372

In [24]:
pc_df.get_expectation_suite(discard_failed_expectations=False)

{
  "expectations": [
    {
      "meta": {},
      "kwargs": {
        "column": "mod",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "patient_id",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "dos",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "procedure_code",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "procedure_code",
        "value_set": [
          "67028"
        ],
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_be_in_set"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "dos

In [25]:
with open( "pc_expectation_file.json", "w") as pc_file:
    pc_file.write(
        json.dumps(pc_df.get_expectation_suite().to_json_dict())
    )

636

In [26]:
dx_df.get_expectation_suite(discard_failed_expectations=False)

{
  "expectations": [
    {
      "meta": {},
      "kwargs": {
        "column": "dx_code",
        "regex": "(?i)(H35.?32)|(362.?52)|(414173003)",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_match_regex"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "dx_code",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "pt_id",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "dos",
        "mostly": 1
      },
      "expectation_type": "expect_column_values_to_not_be_null"
    },
    {
      "meta": {},
      "kwargs": {
        "column": "dos",
        "strftime_format": "%Y-%m-%d"
      },
      "expectation_type": "expect_column_values_to_match_strftime_format"
    },
    {
      "meta": {},
      "kwargs": {
       

In [27]:
with open( "dx_expectation_file.json", "w") as dx_file:
    dx_file.write(
        json.dumps(dx_df.get_expectation_suite().to_json_dict())
    )

630